# Day 18: RAM Run

### Part 1

You and The Historians look a lot more pixelated than you remember. You're inside a computer at the North Pole!

Just as you're about to check out your surroundings, a program runs up to you. "This region of memory isn't safe! The User misunderstood what a pushdown automaton is and their algorithm is pushing whole bytes down on top of us! Run!"

The algorithm is fast - it's going to cause a byte to fall into your memory space once every nanosecond! Fortunately, you're faster, and by quickly scanning the algorithm, you create a list of which bytes will fall (your puzzle input) in the order they'll land in your memory space.

Your memory space is a two-dimensional grid with coordinates that range from 0 to 70 both horizontally and vertically. However, for the sake of example, suppose you're on a smaller grid with coordinates that range from 0 to 6 and the following list of incoming byte positions:

```
5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0
```

Each byte position is given as an X,Y coordinate, where X is the distance from the left edge of your memory space and Y is the distance from the top edge of your memory space.

You and The Historians are currently in the top left corner of the memory space (at 0,0) and need to reach the exit in the bottom right corner (at 70,70 in your memory space, but at 6,6 in this example). You'll need to simulate the falling bytes to plan out where it will be safe to run; for now, simulate just the first few bytes falling into your memory space.

As bytes fall into your memory space, they make that coordinate corrupted. Corrupted memory coordinates cannot be entered by you or The Historians, so you'll need to plan your route carefully. You also cannot leave the boundaries of the memory space; your only hope is to reach the exit.

In the above example, if you were to draw the memory space after the first 12 bytes have fallen (using . for safe and # for corrupted), it would look like this:

```
...#...
..#..#.
....#..
...#..#
..#..#.
.#..#..
#.#....
```

You can take steps up, down, left, or right. After just 12 bytes have corrupted locations in your memory space, the shortest path from the top left corner to the exit would take 22 steps. Here (marked with O) is one such path:

```
OO.#OOO
.O#OO#O
.OOO#OO
...#OO#
..#OO#.
.#.O#..
#.#OOOO
```

Simulate the first kilobyte (1024 bytes) falling onto your memory space. Afterward, what is the minimum number of steps needed to reach the exit?


In [77]:
import com.psyndicate.aoc.*
import java.util.PriorityQueue
import kotlin.test.*

data class Path(val position: Coord, val steps: Int)

fun search(
    corruption: Set<Coord>,
    mazeSize: Int,
    position: Coord,
    goal: Coord
): Int {
    var minSteps = Int.MAX_VALUE
    val seen = mutableMapOf<Coord, Int>()
    val queue = PriorityQueue<Path>(compareBy { it.steps })
    queue.add(Path(position, 0))
    while (queue.isNotEmpty()) {
        queue.addAll(
            queue.poll()
                .takeIf { entry ->
                    val location = entry.position
                    val steps = entry.steps
                    val bestScore = seen.getOrDefault(location, Int.MAX_VALUE)
                    if (steps < bestScore) seen[location] = steps
                    if (location == goal) {
                        if (steps >= minSteps) {
                            // Scores are getting worse
                            return@search minSteps
                        }
                        minSteps = steps
                    }
                    steps < bestScore
                }
                ?.let { entry ->
                    CardinalDirections
                        .map { entry.position + it }
                        .filter { it !in corruption && it.x in 0..mazeSize && it.y in 0..mazeSize }
                        .map { Path(it, entry.steps + 1) }
                }
                ?: emptyList()
        )

    }
    return minSteps
}

fun parseInput(input: String): List<Coord> =
    input.split('\n').map { it.split(',').map { it.toInt() }.let { Coord(it[0], it[1]) } }

fun calculatePart1(input: String, mazeSize: Int, amount: Int): Int =
    search(parseInput(input).take(amount).toSet(), mazeSize, Coord(0, 0), Coord(mazeSize, mazeSize))

val testInput =
    """
    5,4
    4,2
    4,5
    3,0
    2,1
    6,3
    2,4
    1,5
    0,6
    3,3
    2,6
    5,1
    1,2
    5,5
    2,5
    6,5
    1,4
    0,4
    6,4
    1,1
    6,1
    1,0
    0,5
    1,6
    2,0
    """.trimIndent()

val testInputPart1Result = calculatePart1(testInput, 6, 12)
DISPLAY("Test Part 1: ${testInputPart1Result}")
assertEquals(22, testInputPart1Result)

val puzzleInput = readResourceAsText("/day18/input.txt")
val part1Result = calculatePart1(puzzleInput, 70, 1024)
DISPLAY("Part 1: ${part1Result}")
assertEquals(348, part1Result)

Test Part 1: 22

Part 1: 348

### Part 2

The Historians aren't as used to moving around in this pixelated universe as you are. You're afraid they're not going to be fast enough to make it to the exit before the path is completely blocked.

To determine how fast everyone needs to go, you need to determine the first byte that will cut off the path to the exit.

In the above example, after the byte at 1,1 falls, there is still a path to the exit:

```
O..#OOO
O##OO#O
O#OO#OO
OOO#OO#
###OO##
.##O###
#.#OOOO
```

However, after adding the very next byte (at 6,1), there is no longer a path to the exit:

```
...#...
.##..##
.#..#..
...#..#
###..##
.##.###
#.#....
```

So, in this example, the coordinates of the first byte that prevents the exit from being reachable are 6,1.

Simulate more of the bytes that are about to corrupt your memory space. What are the coordinates of the first byte that will prevent the exit from being reachable from your starting position? (Provide the answer as two integers separated by a comma with no other characters.)

In [75]:
fun calculatePart2(input: String, mazeSize: Int): String {
    val corrupted = parseInput(input)
    return (corrupted.size - 1 downTo 0).first { amount ->
        search(corrupted.take(amount).toSet(), mazeSize, Coord(0, 0), Coord(mazeSize, mazeSize)) != Int.MAX_VALUE
    }.let { "${corrupted[it].x},${corrupted[it].y}" }
}

val testInputPart2Result = calculatePart2(testInput, 6)
DISPLAY("Test Part 2: ${testInputPart2Result}")
assertEquals("6,1", testInputPart2Result)

val part2Result = calculatePart2(puzzleInput, 70)
DISPLAY("Part 2: ${part2Result}")
assertEquals("54,44", part2Result)

Test Part 2: 6,1

Part 2: 54,44